In [25]:
URL = 'http://bit.ly/scipyla_scada2iot'

In [26]:
!qr --factory=pymaging $URL > img/qr.png

[![View this Presentation Online](img/qr.png?refresh)](http://bit.ly/scipyla_scada2iot)
View this presentation on Notebook Viewer

# From SCADA to IoT

## Nahuel Defossé

# Disclaimer
 * Not a good English speaker but definetly worse at Protuguese.
 * Some material cannot be distributed openly, approach me personaly to get more details.

# About me <i class="fa fa-info-circle" aria-hidden="true"></i>


 * Chief of Practical Assgnments at Universidad Nacional de la Patagonia
 * Memmber of Research groups focused on "Microcontroadores e Internet", "Web Augmentation on SCADA systems" and "Coalbrative Systems".
 * Python Advocate. It a good hammer 80% of the time.

# Owed credit <i class="fa fa-users" aria-hidden="true"></i>

 *  Ricardo A Lopez
 * Fernando A Tinetti 
 * Lautaro Pecile 
 * van Haaster Diego 
 * Wahler Sebastián

# Where do I come from? <i class="fa fa-location-arrow" aria-hidden="true"></i>


# SCADA <i class="fa fa-tachometer" aria-hidden="true"></i>


## Monitor and control industrial processes


![](img/Scada_BetonSantrali.png)

![](img/scadasystem.img_.png)

# Industrial Processes
<center>
![](img/incredible_machine.gif)
</center>

# Implementing network Protocols with Mara

## Structure

Construct
## Sequence
Twisted

# Protocols

### Open/Closed
 - DNP3
 - Modbus
 - IEC 60870-5

# Mara
<pre style="font-size: .7em;">
┌─────┬─────┬─────┬─────┬─────┬─────┬─────┐┌─────┐┌─────┬─────┐
│ SOF │ SEQ │ QTY │ SRC │ DST │ CMD │ CMD ││ ... ││ BCL │ BCH │
│     │     │     │     │     │     │     ││     ││     │     │
└─────┴─────┴─────┴─────┴─────┴─────┴─────┘└─────┘└─────┴─────┘
┌─────┬───────────┬───────────┬─────┬───────────┬───────────┐  
│ QSV │ SV0       │ SV1       │ QDI │ DI0       │ DI1       │  
│     │           │           │     │           │           │  
└─────┴───────────┴───────────┴─────┴───────────┴───────────┘  
┌─────┬───────────┬───────────┬─────┬─────────────────────────┐
│ QAI │ AI0       │ AI1       │ QEV │ EV..                    .
│     │           │           │     │                         │
└─────┴───────────┴───────────┴─────┴─────────────────────────┘
</pre>

# Implementing Mara

## Structure: Construct

## Sequence: Twisted

# Human Machine Interface (HMI)

<br/>
## Formulas
## SVG

Buying devices made things 

#

# IoT

* More deviced than people, producing data.
* Analytics giving value to data.
* Constrained devices and Smart Devices.
* Standards

# Getting Data

## CoAP
 *  Constrained Application Protocol
 * REST over UDP
 
## MQTT
 * Message Queue Telemetry Transport
 *

**CoAP** inherits many concepts from HTTP <br/>
**MQTT** Message Queues, QoS.

# Integration with Smart Devices

## Kivy

* Easy to develop
* Quite complete Python implementation
* Good documentation


In [13]:
!git push origin master

Counting objects: 13, done.
Delta compression using up to 4 threads.
Compressing objects: 100% (11/11), done.
Writing objects: 100% (13/13), 4.18 KiB | 0 bytes/s, done.
Total 13 (delta 0), reused 0 (delta 0)
To git@github.com:D3f0/talks.git
   15f0d63..7436f93  master -> master


In [ ]:
!git add img && git add -u && git commit -m "Updates $(date)" && git push origin master

[master 697c510] Updates Sun May 15 20:45:21 ART 2016
 7 files changed, 302 insertions(+), 41 deletions(-)
 create mode 100644 2016/SciPyLatam/FromScadaToIoT/img/Scada_BetonSantrali.png
 create mode 100644 2016/SciPyLatam/FromScadaToIoT/img/incredible_machine.gif
 create mode 100644 2016/SciPyLatam/FromScadaToIoT/img/qr.png
 create mode 100644 2016/SciPyLatam/FromScadaToIoT/img/qr.pngs
 create mode 100644 2016/SciPyLatam/FromScadaToIoT/img/scadasystem.img_.png
 create mode 100644 2016/SciPyLatam/FromScadaToIoT/img/tumblr_inline_ns4gu3qIKl1raprkq_540.gif
Counting objects: 13, done.
Delta compression using up to 4 threads.
Compressing objects: 100% (12/12), done.
Writing objects: 100% (13/13), 5.44 MiB | 400.00 KiB/s, done.
Total 13 (delta 0), reused 0 (delta 0)
To git@github.com:D3f0/talks.git
   f37d738..697c510  master -> master


In [ ]:
%%bash
jupyter-nbconvert --to slides From\ Scada\ To\ IoT.ipynb --post serve
# open From\ Scada\ To\ IoT.slides.html

In [6]:
!rm reveal.js

In [8]:
!open custom.css